1. [Откройте файл с данными и изучите общую информацию](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка артефактных значений](#artefact)
    * [Обработка пропусков](#null)
    * [Замена типа данных](#data_type)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemma)
    * [Категоризация данных](#category)
3. [Ответьте на вопросы](#questions)
    * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#q1)
    * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#q2)
    * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#q3)
    * [Как разные цели кредита влияют на его возврат в срок?](#q4)
4. [Общий вывод](#conclusion)
5. [Чек-лист готовности проекта](#check-list)

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка.

**Цель исследования** - проверить гипотезу влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<a id="start"></a>
## Шаг 1. Откройте файл с данными и изучите общую информацию


Импортируем библиотеку pandas:

In [528]:
import pandas as pd

Прочитаем файл data.csv из папки /datasets и сохраним его в переменную df:


In [529]:
df = pd.read_csv('/datasets/data.csv')

Получаем первые 5 строк таблицы df:

In [530]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


 Получим информацию о таблице:

In [531]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Названия всех столбцов приведены к змеиному регистру и не нуждаются в корректировке.

Согласно документации к данным: 
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Количество значений в столбцах различаются. Значит, в данных есть пропущенные значения.

**Вывод**

Исходная таблица df содержит информацию о клиентах банка, их возрасте, уровне образования и т.д. Основными столбцами для проверки гипотезы могут быть:
- семейное положение(family_status);
- количество детей(children);
- наличие задолженности по возврату кредитов(debt).


<a id="preprocessing"></a>
## Шаг 2. Предобработка данных

<a id="artefact"></a>
### Обработка артефактных значений

Взглянув на таблицу, можно увидеть что почти все значения в столбце с количеством дней общего трудового стажа имеют отрицательное значение. Применим функцию `abs()` к этому столбцу: 

In [532]:
df['days_employed'] = abs(df['days_employed'])

Заново выведем часть таблицы на экран, чтобы убедиться, что функция сработала:

In [533]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<a id="null"></a>
### Обработка пропусков

Узнаем количество пропущенных значений в таблице:

In [534]:
missing_items = df.isna().sum()
missing_items

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущены значения в столбцах days_employed и total_income.
Причины пропусков в столбцах могут быть следующие: 
 - банк не запрашивал эти данные у клиента,
 - "человеческий фактор",  так как общий трудовой стаж в днях почти во всех строках с отрицательным знаком, возможно, в таблице не одна ошибка,
 - технически данные не верно выгружаются, что-то не работает так, как должно;
 - клиент трудоустроен неофициально и поэтому просто не может предоставить банку такую информацию,
 - для выдачи определенного вида кредита банку эта информация не требуется.

Пропущенные значения в столбцах не сильно повлияют на результаты исследования, так как:
- находятся не в ключевых столбцах;
- доля пропущенных значений не критична и скорее всего не повлияет на результат анализа.

Заменим их явными обозначениями. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и выполним замену пропущенных значений:

In [535]:
columns_to_replace = ['days_employed','total_income']
for column in columns_to_replace:
    df[column] = df[column].fillna('0')

Убедимся, что в таблице не осталось пропусков. Еще раз считаем пропущенные значения:

In [536]:
missing_items = df.isna().sum()
missing_items

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

В исходной таблице имелись столбцы с пропущенными значениями, которые могли появиться из-за технической ошибки или по другой иной причине. Пропущенные значения заполнены значением `'0'` и скорее всего не изменят результаты исследования.

<a id="data_type"></a>
### Замена типа данных

Заменим вещественный тип данных `float64` на целочисленный `int64`. Для этого применим метод `astype()`:

In [537]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

Снова выведем информацию о таблице на экран:

In [538]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Заменили вещественный тип данных на целочисленный методом `astype()`,потому что применить его удобнее всего. Необходимые столбцы в таблице после замены пропущенных значений относятся к типу float64 и не требуют применения других методов замены типа данных, таких как `to_numeric()`.

Проверим все необходимые нам для исследования столбцы на предмет аномальных значений:


In [539]:
display(df['children'].unique())
display(df['family_status'].unique())
display(df['debt'].unique())
display(df['purpose'].unique())
display(df['education'].unique())
display(df['income_type'].unique())

array([ 1,  0,  3,  2, -1,  4, 20,  5])

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array([0, 1])

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Приведем столбцы к единому регистру:


In [540]:
df['education_lower'] = df['education'].str.lower()
df['family_status_lower'] = df['family_status'].str.lower()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,family_status_lower
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,женат / замужем
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,женат / замужем
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,женат / замужем
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,женат / замужем
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,гражданский брак


Проверим аномальные значения в столбце с количеством детей:

In [541]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Видим такие аномальные значений как -1 и 20.
Посмотрим в каких строчках и в каком количестве присутствуют аномальные значения:

In [542]:
df.query('children == -1 or children == 20')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,family_status_lower
291,-1,4417,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование,среднее,гражданский брак
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья,среднее,женат / замужем
705,-1,902,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля,среднее,женат / замужем
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости,среднее,женат / замужем
742,-1,3174,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование,среднее,женат / замужем
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21140,-1,1422,44,среднее,1,женат / замужем,0,F,компаньон,0,169562,операции со своей недвижимостью,среднее,женат / замужем
21325,20,601,37,среднее,1,женат / замужем,0,F,компаньон,0,102986,профильное образование,среднее,женат / замужем
21390,20,0,53,среднее,1,женат / замужем,0,M,компаньон,0,0,покупка жилой недвижимости,среднее,женат / замужем
21404,20,494,52,среднее,1,женат / замужем,0,M,компаньон,0,156629,операции со своей недвижимостью,среднее,женат / замужем


In [543]:
f'Аномальные значения есть в 123 строках из 21525, что составляет всего {123/21525:.2%} от общего числа строк' 


'Аномальные значения есть в 123 строках из 21525, что составляет всего 0.57% от общего числа строк'

Предположим, что удаление этих строк никак не повлияет на результаты исследования, так как их количество менее 1% от общего количества. Выберем строки, где втречается аномальное значение и удалим их, затем обновим индексы в таблице:

In [544]:
df_children_anomaly = df.query('children == -1 or children == 20') 
df = df.drop(df_children_anomaly.index, axis = 0)
df.reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,family_status_lower
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,женат / замужем
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,женат / замужем
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,женат / замужем
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,женат / замужем
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,гражданский брак
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21397,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,среднее,гражданский брак
21398,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,среднее,женат / замужем
21399,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,среднее,гражданский брак
21400,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,среднее,женат / замужем


Запишем в новую переменную `df1` таблицу с нужными для исследования столбцами

In [545]:
columns = ['children','education_lower','family_status_lower','family_status_id','debt','total_income','purpose']
df1 = pd.DataFrame(df, columns=columns)
df1.reset_index(drop=True).head()

,children,education_lower,family_status_lower,family_status_id,debt,total_income,purpose
0,1,высшее,женат / замужем,0,0,253875,покупка жилья
1,1,среднее,женат / замужем,0,0,112080,приобретение автомобиля
2,0,среднее,женат / замужем,0,0,145885,покупка жилья
3,3,среднее,женат / замужем,0,0,267628,дополнительное образование
4,0,среднее,гражданский брак,1,0,158616,сыграть свадьбу


<a id="duplicates"></a>
### Обработка дубликатов

Посчитаем явные дубликаты в таблице:

In [546]:
duplicated_df = df1.duplicated().sum()
duplicated_df

1430

Удалим явные дубликаты (удалим старые индексы и сформируем новые):

In [547]:
df = df.drop_duplicates().reset_index(drop=True)

Снова посчитаем дубликаты и убедимся, что избавились от них:

In [548]:
duplicated_df = df.duplicated().sum()
duplicated_df

0

**Вывод**

Для поиска дубликатов воспользовались методом `duplicated() и sum()` , для удаления дубликатов использовался метод `drop_duplicates()`. 
Дубликаты в данных могли появиться по техническим причинам, данные "задвоились".

<a id="lemma"></a>
### Лемматизация

Для лемматизации импортируем библиотеку `pymystem3`:

In [549]:
from pymystem3 import Mystem
m = Mystem()

Выведем список всех значений столбца с целью получения кредита, для дальнейшей лемматизации:

In [550]:
purpose = df1['purpose'].sort_values()
display(purpose)

5055          автомобили
1454          автомобили
15416         автомобили
18165         автомобили
10224         автомобили
              ...       
7392     сыграть свадьбу
18842    сыграть свадьбу
10997    сыграть свадьбу
4718     сыграть свадьбу
8278     сыграть свадьбу
Name: purpose, Length: 21402, dtype: object

Лемматизируем слова из переменной `purpose` и запишем результат в переменную `lemmas`:

In [551]:
lemmas = m.lemmatize(" ".join(purpose))

Посчитаем сколько раз каждое слово употребляется в столбце purpose, с помощью метода `most_common()` отсортируем список по убыванию:

In [552]:
from collections import Counter
lemma = Counter(lemmas)
lemma.most_common()

[(' ', 54886),
 ('недвижимость', 6330),
 ('покупка', 5880),
 ('жилье', 4450),
 ('автомобиль', 4288),
 ('образование', 3997),
 ('с', 2906),
 ('операция', 2593),
 ('свадьба', 2337),
 ('свой', 2224),
 ('на', 2218),
 ('строительство', 1870),
 ('высокий', 1368),
 ('получение', 1311),
 ('коммерческий', 1307),
 ('для', 1291),
 ('жилой', 1225),
 ('подержать', 956),
 ('сделка', 939),
 ('заниматься', 908),
 ('дополнительный', 902),
 ('проведение', 772),
 ('сыграть', 769),
 ('сдача', 651),
 ('семья', 640),
 ('собственный', 628),
 ('со', 626),
 ('ремонт', 609),
 ('приобретение', 460),
 ('профильный', 432),
 ('\n', 1)]

**Вывод**

Получили леммы в значениях столбца с целями получения кредита. 
Посмотрев на отсортированный список, можно сделать вывод, что имеется несколько основных целей получения кредита:
- покупка недвижимости/жилья,
- покупка автомобиля,
- получение образования.

Следом можно выделить такие цели как:

- свадьба,
- строительство.

<a id="category"></a>
### Категоризация данных

**Категоризация данных. Количество детей**

Категоризируем клиентов по количеству детей.

In [553]:
df1['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Аномальных значений нет, значит распределим клиентов так:
- Клиенты без детей попадают в категорию "бездетная",
- Клиенты с 1-2 детьми попадут в категорю "семья с детьми",
- Клиенты с 3 детьми и более будут считаться "многодетная".

Запишем правила как функцию и протестируем ее.

In [554]:
def children_amount(children):
    if children <= 0:
        return 'бездетная'
    if 1<= children <=2:
        return 'семья с детьми'
    if children >= 3:
        return 'многодетная'

In [555]:
display(children_amount(0))
display(children_amount(1))
display(children_amount(15))

'бездетная'

'семья с детьми'

'многодетная'

Написанная функция работает корректно, добавим отдельный столбец в таблицу, применим метод `apply()`.

In [556]:
df1['children_amount'] = df1['children'].apply(children_amount)

Выведем статистику по количеству детей:

In [557]:
df1['children_amount'].value_counts()

бездетная         14149
семья с детьми     6873
многодетная         380
Name: children_amount, dtype: int64

**Категоризация данных. Ежемесячный доход**

Категоризируем столбец с информацией о ежемесячном доходе клиентов. Будем считать, что низкий достаток у людей с доходом 25 тысяч и менее, от 25 до 100 - средний, от 100 до 500 - высокий, от 500 и более - богачи.

In [558]:
def total_income_def(total_income):
    
    if total_income <= 25000:
        return 'низкий достаток'
    if 25001 <= total_income <= 100000:
        return 'средний достаток'
    if 100001 <= total_income <= 500000:
        return 'высокий достаток'
    if total_income >= 500001:
        return 'богачи'
    
total_income_def(155555)

'высокий достаток'

Функция работает корректно, применим ее к столбцу `total_income` и добавим в таблицу:

In [559]:
df1['income_level'] = df1['total_income'].apply(total_income_def)

Посчитаем значения в новом столбце:

In [560]:
df1['income_level'].value_counts()

высокий достаток    14574
средний достаток     4436
низкий достаток      2170
богачи                222
Name: income_level, dtype: int64

**Категоризация данных. Цель получения кредита**

Для того, чтобы категоризировать данные столбца "Цель получения кредита" нужно вспомнить какие основные цели мы выделили в леммах: недвижимость, жилье, автомобиль, образование, свадьба и строительство. Запишем их в функцию, проверим ее работу:

In [561]:
def purpose_def(purpose):
        if 'авто' in purpose:
            return 'Цель - автомобиль'
        if 'недвиж'in purpose:
            return 'Цель - недвижимость'
        if 'жиль' in purpose:
            return 'Цель - недвижимость'
        if 'образовани' in purpose:
            return 'Цель - образование'
        if 'свадьб' in purpose:
            return 'Цель - свадьба'
        if 'строительств' in purpose:
            return 'Цель - строительство'
        return 'Не могу определить категорию'
    
display(purpose_def('сделка с подержанным автомобилем'))
display(purpose_def('операции с жильем'))
display(purpose_def('абракадабра'))

'Цель - автомобиль'

'Цель - недвижимость'

'Не могу определить категорию'

Функция работает корректно, применим ее ко всему столбцу `purpose`:

In [562]:
df1['purpose_category'] = df1['purpose'].apply(purpose_def)

Посчитаем значения в новом столбце:

In [563]:
df1['purpose_category'].value_counts()

Цель - недвижимость    10780
Цель - автомобиль       4288
Цель - образование      3997
Цель - свадьба          2337
Name: purpose_category, dtype: int64

**Вывод**

Проведя категоризацию имеющихся данных можно сказать, что:
- преимущественное количество клиентов не имеет детей, чуть больше четверти имеет 1-2 детей и меньше всего многодетных клиентов,
- большая часть клиентов имеет высокий достаток в месяц (от 100 000 до 500 000 т.р.),
- основная цель получения кредита - различные операции с недвижимостью и жильем.

<a id="questions"></a>
## Шаг 3. Ответьте на вопросы

<a id="q1"></a>
### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построим и выведем сводную таблицу, отображающую зависимость между наличием детей и возвратом кредита в срок. Для этого воспользуемся методом `pivot_table()`.
Для четкого понимания взаимосвязи выведем в новый столбец процент не вернувшихся кредитов:

In [564]:
pivot_children = df1.pivot_table(index=['children_amount'],columns='debt',values='children',aggfunc='count')
pivot_children_index = pivot_children.reset_index()
pivot_children_index.set_axis(['children_amount','yes','no'],'columns',inplace=True)
pivot_children_index['no_return'] = pivot_children_index['no']/pivot_children_index[['yes','no']].sum(axis=1)*100
pivot_children_index

,children_amount,yes,no,no_return
0,бездетная,13086,1063,7.512898
1,многодетная,349,31,8.157895
2,семья с детьми,6235,638,9.282700


**Вывод**

Проанализировав таблицу и дополнительные расчеты можно сказать, что :
- семьи без детей реже не возвращают банку кредиты в срок, то есть могут быть более надежными клиентами,
- семьи с 1 или 2 детьми чаще остальных не возвращают кредиты в срок.


<a id="q2"></a>
### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим и выведем сводную таблицу, отображающую зависимость между семейным положением и возвратом кредита в срок. Для этого воспользуемся методом `pivot_table()`

In [565]:
pivot_family = df1.pivot_table(index=['family_status_lower'],columns='debt',values='family_status_id',aggfunc='count')
pivot_family_index = pivot_family.reset_index()
pivot_family_index.set_axis(['family_status_lower','yes','no'],'columns',inplace=True)
pivot_family_index['no_return'] = pivot_family_index['no']/pivot_family_index[['yes','no']].sum(axis=1)*100
pivot_family_index

,family_status_lower,yes,no,no_return
0,в разводе,1105,84,7.064760
1,вдовец / вдова,889,63,6.617647
2,гражданский брак,3775,385,9.254808
3,женат / замужем,11375,927,7.535360
4,не женат / не замужем,2526,273,9.753483


**Вывод**

Проанализировав сводную таблицу, отвечая на вопрос, можно сказать, что:
- самые ответственные клиенты банка вдовы и вдовцы, они реже остальных не возвращают кредиты в срок, 
- так же редко не возвращают кредиты в срок клиенты в разводе и в браке,
- чаще всего остальных не возвращают кредиты в срок клиенты в гражданском браке и одинокие люди.

<a id="q3"></a>
### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построим и выведем сводную таблицу, отображающую зависимость между уровнем дохода и возвратом кредита в срок:

In [566]:
pivot_income = df1.pivot_table(index=['income_level'],columns='debt',values='total_income',aggfunc='count')
pivot_income_index = pivot_income.reset_index()
pivot_income_index.set_axis(['income_level','yes','no'],'columns',inplace=True)
pivot_income_index['no_return'] = pivot_income_index['no']/pivot_income_index[['yes','no']].sum(axis=1)*100
pivot_income_index

,income_level,yes,no,no_return
0,богачи,208,14,6.306306
1,высокий достаток,13380,1194,8.192672
2,низкий достаток,1999,171,7.880184
3,средний достаток,4083,353,7.957619


**Вывод**

Проанализировав сводную таблицу зависимости между уровнем дохода и возврата кредита в срок, можно сказать, что:
- реже всего не возвращают кредит в срок очень богатые люди,
- чаще всего задерживают выплату по кредиту люди с высоким и средним достатком.

<a id="q4"></a>
### Как разные цели кредита влияют на его возврат в срок?

Построим и выведем сводную таблицу, отображающую зависимость от цели получения кредита и его возврат в срок:

In [567]:
pivot_purpose = df1.pivot_table(index=['purpose_category'],columns='debt',values='purpose',aggfunc='count')
pivot_purpose_index = pivot_purpose.reset_index()
pivot_purpose_index.set_axis(['purpose_category','yes','no'],'columns',inplace=True)
pivot_purpose_index['no_return'] = pivot_purpose_index['no']/pivot_purpose_index[['yes','no']].sum(axis=1)*100
pivot_purpose_index

,purpose_category,yes,no,no_return
0,Цель - автомобиль,3888,400,9.328358
1,Цель - недвижимость,10000,780,7.235622
2,Цель - образование,3628,369,9.231924
3,Цель - свадьба,2154,183,7.830552


**Вывод**

Проанализировав информацию о взаимосвязи цели кредита и его возврата в срок, можно сказать: 
- если клиент берет кредит на различные цели связанные с автомобилем или образованием, он с большей вероятностью задержит выплату по кредиту, 
- клиенты, взявшие кредит на свадьбу и различные операции с недвижимостью, чаще других выплачивают кредиты в срок. 

<a id="conclusion"></a>
## Шаг 4. Общий вывод

Целью исследования было определить взаимосвязь наличия детей и семейного положения на возврат кредита в срок. Для этого проделали следующие шаги: 
- обработали дубликаты и пропуски,
- заменили типы данных в нужных столбцах,
- выделили леммы в столбце с целью получения кредита,
- категоризировали данные. 

Отвечая на основные вопросы исследования можно сказать, что: 
- можно проследить небольшую зависимость между наличием детей и возвратом кредита в срок. Семьи с 1-2 детьми чаще остальных не возвращают кредиты в срок.
- имеется связь между семейным положением и возвратом кредита в срок, самые "ненадежные" клиенты - клиенты состоящие в гражданском браке и одинокие люди.
- отследить четкую взаимосвязь между уровнем дохода и вернувшимся кредитом в срок сложнее. Самые ответственные клиенты - клиенты с очень высоким достатком(от 500 000 и более). Клиенты с высоким достатком чаще остальных не возвращают кредиты в срок.
- обращаясь к информации о цели выдачи кредита можно сказать, что клиенты, взявшие кредит на получение образования и автомобиль, с большей вероятносью вернут его в срок, чем те, кто взял кредит на свадьбу или на различные операции с недвижимостью. 